## Experiment 3: Gated Recurrent Unit (GRU) Model with TensorFlow TextVectorization Layer Embeddings
**(Version 1)**

**1. Import the necessary libraries and modules for this experiment**

In [1]:
import datetime
from packaging import version
from collections import Counter
import numpy as np
import pandas as pd
import time
import os
import re
import string

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import nltk
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
# %pip install tensorflow_datasets
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.backend as k
from sklearn.model_selection import train_test_split

In [2]:
# Set the default precision for numpy
np.set_printoptions(precision=3, suppress=True)

# Enable display of multiple outputs per Jupyter Notebook cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

---

**2. Load in the golf course reviews dataset & create a new label column**


In [3]:
file_path = "top_and_non_golf_course_reviews.csv"
df = pd.read_csv(file_path)

# Create a new label column that indicates whether the review is a top100 course or not
df['top100'] = df['label'].apply(lambda x: 1 if x == 'top100' else 0)

---

**3. Split the dataset into training, validation, and testing sets**

In [4]:
train_df, remaining = train_test_split(df, test_size=0.33, stratify=df['top100'], random_state=42)
val_df, test_df = train_test_split(remaining, test_size=0.5, stratify=remaining['top100'], random_state=42)

# Check the shape of the training, validation, and test sets
print(f"Training Dataset Shape: {train_df.shape}")
print(f"Validation Dataset Shape: {val_df.shape}")
print(f"Test Dataset Shape: {test_df.shape}")

Training Dataset Shape: (80, 11)
Validation Dataset Shape: (20, 11)
Test Dataset Shape: (20, 11)


----

**4. Convert the split DataFrames into TensorFlow Datasets**

In [5]:
train_ds = tf.data.Dataset.from_tensor_slices(dict(train_df))
val_ds = tf.data.Dataset.from_tensor_slices(dict(val_df))
test_ds = tf.data.Dataset.from_tensor_slices(dict(test_df))

----

**5. Create `custom_stopwords` function and `text_vectorization` layer**

In [6]:
# Define a `custom_stopwords` function to remove stopwords, strip punctuation, and lowercase the text
def custom_stopwords(input_text):
    """
    Removes stopwords, strips punctuation, and lowers the input text.

    Args:
        input_text (tf.Tensor): The input text to be processed.

    Returns:
        tf.Tensor: The processed text with stopwords removed, punctuation stripped, and lowercased.
    """
    lowercase = tf.strings.lower(input_text)
    stripped_punct = tf.strings.regex_replace(lowercase,
                                            '[%s]' % re.escape(string.punctuation),
                                            '')
    return tf.strings.regex_replace(stripped_punct, r'\b(' + r'|'.join(STOPWORDS) + r')\b\s*', "")

# Download stopwords from the NLTK library
nltk.download('stopwords', quiet=True)
STOPWORDS = stopwords.words("english")

# Define the maxium sequence and token length for this experiment
max_length =  3073
max_tokens = 10000

# Create a TextVectorization layer
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
    standardize=custom_stopwords
)

True

----

**6. Adapt the TextVectorization layer to a text_only_dataset of the training dataset**

In [7]:
# Create a text_only_train_dataset which contains only the review_text column of the training dataset
text_only_train_dataset = train_ds.map(lambda x: x['review_text'])

# Adapt the TextVectorization layer to the text_only_train_dataset
text_vectorization.adapt(text_only_train_dataset)

# Create int_train_ds, int_val_ds, and int_test_ds from the train, val, and test datasets respectively using the TextVectorization layer
int_train_ds = train_ds.map(lambda x: (text_vectorization(x['review_text']), x['top100']))
int_val_ds = val_ds.map(lambda x: (text_vectorization(x['review_text']), x['top100']))
int_test_ds = test_ds.map(lambda x: (text_vectorization(x['review_text']), x['top100']))

# Batch and pad the datasets to have a sequence lenfth dimension
batch_size = 32
max_sequence_length = 3073

int_train_ds = int_train_ds.map(lambda x, y: (x, y)).padded_batch(batch_size, padded_shapes=(max_sequence_length, ()))
int_val_ds = int_val_ds.map(lambda x, y: (x, y)).padded_batch(batch_size, padded_shapes=(max_sequence_length, ()))
int_test_ds = int_test_ds.map(lambda x, y: (x, y)).padded_batch(batch_size, padded_shapes=(max_sequence_length, ()))

2024-08-10 11:26:36.982598: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


----
**7. Create Two Layer GRU Model**

- Two GRU layers (64 units and 32 units)
- ReLu activation function
- RMSprop optimizer (learning rate = 0.001)
- Vocabulary size = 10000

In [8]:
# Define the model constants
vocab_size = 10000
embedding_dim = 356

# Build the GRU model
inputs = tf.keras.Input(shape=(None,), dtype="int64", name="input")
embedding = layers.Embedding(input_dim=vocab_size,
                             output_dim=embedding_dim,
                             mask_zero=True, name="embedding")(inputs)
x = layers.GRU(64, return_sequences=True, name="gru_1")(embedding)
x = layers.GRU(32, name="gru_2")(x)
outputs = layers.Dense(1, activation="sigmoid", name="output")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name="Two_Layer_GRU_TF-TV")

---- 
**8. Compile the model**

In [9]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=["accuracy"])

# Display the model summary
model.summary()

Model: "Two_Layer_GRU_TF-TV"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, None)      │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 356) │  3,560,000 │ input[0][0]       │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, None)      │          0 │ input[0][0]       │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_1 (GRU)         │ (None, None, 64)  │     81,024 │ embedding[0][0],  │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_2 (GRU)         │ (None, 32)        │      9,408 │ gru_1[0][0],      │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         33 │ gru_2[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,650,465 (13.93 MB)

 Trainable params: 3,650,465 (13.93 MB)

 Non-trainable params: 0 (0.00 B)

---
**9. Train the model**

In [10]:
# Clear any existing models in memory
tf.keras.backend.clear_session()

# Define the callbacks for the model training
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("EXP_3_GRU_TF_TV.keras", save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
]

# Train the model
start_time = time.time()
history = model.fit(int_train_ds,
                    validation_data=int_val_ds,
                    epochs=20,
                    callbacks=callbacks)
end_time = time.time()
training_time = end_time - start_time

# Print the training time
print(f"Training Time: {training_time} seconds")

Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.4656 - loss: 0.6931 - val_accuracy: 0.4500 - val_loss: 0.6957
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.7844 - loss: 0.6711 - val_accuracy: 0.4000 - val_loss: 0.7000
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 0.8586 - loss: 0.6390 - val_accuracy: 0.5000 - val_loss: 0.7109
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.8852 - loss: 0.5775 - val_accuracy: 0.5000 - val_loss: 0.7385
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 3s/step - accuracy: 0.9055 - loss: 0.4690 - val_accuracy: 0.4500 - val_loss: 0.7910
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 0.9258 - loss: 0.3145 - val_accuracy: 0.4500 - val_loss: 0.8525
Training Time: 46.4211802482605 seconds


---
**10. Evaluate the model on the test set**

In [11]:
# Load the best model
model = tf.keras.models.load_model("EXP_3_GRU_TF_TV.keras")

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(int_test_ds)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Get the training loss, validation loss, training accuracy, and validation accuracy from the history object
training_loss = history.history['loss'][-1] # the -1 index gets the last epoch
validation_loss = history.history['val_loss'][-1]
training_accuracy = history.history['accuracy'][-1]
validation_accuracy = history.history['val_accuracy'][-1]

"""
1.2.2 Extract the training history and add all evaluation metrics into a history DataFrame
"""
# Extract the training history into a pandas DataFrame
history_df = pd.DataFrame({
    'EXP': [3],
    'Model': ['GRU w/ TextVectorization Embeddings'],
    'Training Loss': [training_loss],
    'Training Accuracy': [training_accuracy],
    'Validation Loss': [validation_loss],
    'Validation Accuracy': [validation_accuracy],
    'Test Loss': [test_loss],
    'Test Accuracy': [test_accuracy],
    'Training Time': [training_time]
})

# Inspect the history DataFrame
history_df

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 602ms/step - accuracy: 0.7000 - loss: 0.6735
Test Loss: 0.6734539270401001, Test Accuracy: 0.699999988079071


'\n1.2.2 Extract the training history and add all evaluation metrics into a history DataFrame\n'

,EXP,Model,Training Loss,Training Accuracy,Validation Loss,Validation Accuracy,Test Loss,Test Accuracy,Training Time
0,3,GRU w/ TextVectorization Embeddings,0.29904,0.9375,0.852489,0.45,0.673454,0.7,46.42118
